In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import HTML
import matplotlib
import numpy as np

In [2]:
df_verhuurd = pd.read_csv('C:\\Users\\jniens\\GoogleDrive\\Python_Projects\\real_estate_analysis\\data_files\\csv\\verhuurde_woningen_driemaanden.csv', sep=';')
df_verkocht = pd.read_csv('C:\\Users\\jniens\\GoogleDrive\\Python_Projects\\real_estate_analysis\\data_files\\csv\\verkochte_woningen_driemaanden.csv', sep=';')
df_stadsdelen = pd.read_csv('C:\\Users\\jniens\\GoogleDrive\\Python_Projects\\real_estate_analysis\\data_files\\csv\\dim_stadsdelen3.csv', sep=';')

In [3]:
df_verkocht['Huisnr.'] = df_verkocht['Huisnr.'].fillna(0).astype(int).astype(str)
df_verkocht['Toev.'] = df_verkocht['Toev.'].fillna('')

In [4]:
df_verkocht['id'] = df_verkocht.Straat + df_verkocht['Huisnr.'].astype(int).astype(str) #+ df_verkocht['Toev.']

In [5]:
cols = ['id', 'Postcode', 'Makelaar', 'Huidige prijs', 'Koop conditie', 'Prijs per m2', 
        'Transactie prijs / huur', 'Transactieprijs per m2', 'Huidige huur', 'Huur conditie', 'Datum aanmelding',
       'Transactiedatum', 'Aantal kamers', 'Soort OG', 'Soort appartement', 'Status']
df_verkocht_subset = df_verkocht[cols]

In [6]:
"Extract number from Postcode column"
df_verkocht_subset.loc[:, 'Postcode_cijfers'] = df_verkocht_subset.Postcode.str[:4].astype(int).copy()

c:\users\jniens\googledrive\python_projects\real_estate_analysis\real_estate_venv\lib\site-packages\pandas\core\indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
c:\users\jniens\googledrive\python_projects\real_estate_analysis\real_estate_venv\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [7]:
col_name_dict = {'Huidige prijs' : 'huidige_prijs_number', 
                 'Prijs per m2' : 'prijs_per_m2_number' ,
                'Transactie prijs / huur' : 'transactieprijs_huur_number', 
                 'Huidige huur' : 'huidige_huur_number',
                'Transactieprijs per m2' : 'transactieprijs_per_m2_number'}

In [8]:
"Convert euro column to type number"
for cur_col, new_col in col_name_dict.items():
    
    df_verkocht_subset[new_col] = df_verkocht_subset[cur_col].str.replace('€', '') \
    .str.replace(',', '')
    df_verkocht_subset[new_col] = pd.to_numeric(df_verkocht_subset[new_col], errors='coerce')


c:\users\jniens\googledrive\python_projects\real_estate_analysis\real_estate_venv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
c:\users\jniens\googledrive\python_projects\real_estate_analysis\real_estate_venv\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
df_stadsdelen.head()

,postcode,stadsdeel
0,1011,Centrum
1,1012,Centrum
2,1013,Centrum
3,1014,Centrum
4,1015,Centrum


In [10]:
df_verkocht_subset = df_verkocht_subset.merge(df_stadsdelen, left_on='Postcode_cijfers', right_on='postcode', how='left')

In [11]:
df_verkocht_groupby = df_verkocht_subset.groupby(['stadsdeel', 'Soort OG'])['transactieprijs_per_m2_number'].agg(['count', 'mean'])

In [12]:
df_verkocht_groupby.unstack()

count                  mean             
Soort OG                  Appartement Woonhuis  Appartement     Woonhuis
stadsdeel                                                               
Centrum                         344.0     47.0  7481.273256  7139.446809
De Aker (Jay)                     6.0     15.0  3538.000000  3536.266667
Nieuw-West                      224.0     74.0  4427.156250  4049.337838
Noord                           188.0    121.0  4399.824468  4753.512397
O IJburg                         44.0     40.0  4376.704545  4716.075000
Oost                            371.0      6.0  6027.547170  6895.166667
Oostelijke Eilanden (Jay)        54.0     13.0  5669.240741  5646.615385
West                            641.0     12.0  6643.469579  7418.666667
ZO Driemand                       NaN      4.0          NaN  3721.750000
Zuid                            432.0     24.0  7343.986111  8312.125000
Zuid Buitenveldert               82.0     19.0  5796.121951  5040.684211
Zuid-Oost                       118.0     39.0  3423.508475  3150.615385

In [13]:
df_verkocht_subset.groupby(['stadsdeel'])['transactieprijs_per_m2_number'].agg(['mean'])

,mean
stadsdeel,
Centrum,7440.184143
De Aker (Jay),3536.761905
Nieuw-West,4333.335570
Noord,4538.323625
O IJburg,4538.309524
Oost,6041.355438
Oostelijke Eilanden (Jay),5664.850746
West,6657.715161
ZO Driemand,3721.750000
